# Chapter 28: Application Security Patterns

Securing database access extends far beyond authentication and encryption. This chapter addresses application-layer security—protecting against injection attacks, implementing fine-grained access controls, designing secure multi-tenant architectures, and ensuring sensitive data remains protected even from privileged users. These patterns satisfy regulatory requirements (GDPR, CCPA, HIPAA, PCI-DSS) while maintaining application performance and developer productivity.

---

## 28.1 SQL Injection Prevention and Secure Query Patterns

SQL injection remains one of the most critical and prevalent database vulnerabilities. PostgreSQL provides robust mechanisms to eliminate this attack vector entirely when used correctly.

### 28.1.1 Parameterized Queries (Prepared Statements)

**The Golden Rule**: Never concatenate user input into SQL strings. Always use parameterized queries where the database driver handles escaping and type conversion.

**Vulnerable Pattern (Never Use)**:

```python
# ANTI-PATTERN: String concatenation
user_input = request.args.get('username')
# Attacker input: ' OR '1'='1' --
query = f"SELECT * FROM users WHERE username = '{user_input}'"
# Results in: SELECT * FROM users WHERE username = '' OR '1'='1' --
# Returns all users, authentication bypassed
```

**Secure Pattern (Industry Standard)**:

```python
# Python (psycopg2/psycopg3)
import psycopg2
from psycopg2.extras import RealDictCursor

conn = psycopg2.connect(DATABASE_URL)
cur = conn.cursor(cursor_factory=RealDictCursor)

# Safe: Parameters passed separately from command
username = request.args.get('username')
cur.execute(
    "SELECT user_id, username, email FROM users WHERE username = %s",
    (username,)  # Tuple of parameters
)
user = cur.fetchone()

# Multiple parameters
cur.execute(
    "SELECT * FROM orders WHERE user_id = %s AND status = %s AND created_at > %s",
    (user_id, status, start_date)
)
```

**Node.js (pg library)**:

```javascript
// Secure parameterized query
const userId = req.query.user_id;
const status = req.query.status;

const result = await pool.query(
  'SELECT * FROM orders WHERE user_id = $1 AND status = $2',
  [userId, status]  // Parameters passed as array
);

// Dangerous (do not use)
const query = `SELECT * FROM orders WHERE user_id = '${userId}'`;
```

**Go (database/sql)**:

```go
// Secure
rows, err := db.Query(
    "SELECT * FROM users WHERE email = $1 AND active = $2",
    email, true
)

// NEVER do this
query := fmt.Sprintf("SELECT * FROM users WHERE email = '%s'", email)
```

**How Parameters Work**:

1. **Parse Phase**: PostgreSQL parses the SQL statement with placeholders (`$1`, `$2`, or `%s` depending on driver)
2. **Plan Phase**: Query planner generates execution plan based on parameter types (not values)
3. **Execute Phase**: Parameters are sent as binary protocol data, completely separate from SQL text
4. **Type Safety**: PostgreSQL casts parameters to declared column types; type mismatches result in errors, not injections

### 28.1.2 Dynamic SQL Safety (Identifier Escaping)

When dynamic table/column names are unavoidable (admin interfaces, ETL systems), use proper identifier quoting.

```python
# Python: psycopg2.sql module for identifiers
from psycopg2 import sql

table_name = user_input  # Could be "users"; DROP TABLE users; --
column_name = "email"

# Safe composition
query = sql.SQL("SELECT {} FROM {} WHERE id = %s").format(
    sql.Identifier(column_name),  # Escapes: becomes "email"
    sql.Identifier(table_name)    # Escapes: becomes "users"; DROP TABLE users; -- (invalid identifier)
)

# If table_name contains malicious content, sql.Identifier raises:
# psycopg2.errors.SyntaxError: syntax error at or near ";"

cur.execute(query, (user_id,))
```

**PostgreSQL Function for Safe Dynamic SQL**:

```sql
-- Safe dynamic query construction inside database
CREATE OR REPLACE FUNCTION get_user_data(
    p_table_name TEXT,
    p_user_id BIGINT
) RETURNS TABLE (id BIGINT, email TEXT) AS $$
DECLARE
    v_sql TEXT;
BEGIN
    -- Validate table name against whitelist (defense in depth)
    IF p_table_name NOT IN ('users', 'customers', 'vendors') THEN
        RAISE EXCEPTION 'Invalid table name: %', p_table_name;
    END IF;
    
    -- Use quote_ident for identifiers, quote_literal for strings
    v_sql := format('SELECT user_id, email FROM %I WHERE user_id = %L',
                   p_table_name, p_user_id);
    
    RETURN QUERY EXECUTE v_sql;
END;
$$ LANGUAGE plpgsql;
```

**Key Functions**:
- `quote_ident(text)`: Escapes identifiers (table/column names), adds double quotes if needed
- `quote_literal(text)`: Escapes string literals, adds single quotes, handles embedded quotes
- `format(spec, ...)` with `%I` (identifier) and `%L` (literal) specifiers

### 28.1.3 ORM Security Patterns

Object-Relational Mappers (ORMs) provide injection protection but introduce their own risks when used improperly.

**Safe Patterns (SQLAlchemy)**:

```python
from sqlalchemy import text
from sqlalchemy.orm import Session

# Safe: Bound parameters
result = session.query(User).filter(User.username == user_input).first()

# Safe: Explicit text() with parameters
result = session.execute(
    text("SELECT * FROM users WHERE username = :username"),
    {"username": user_input}
)

# DANGEROUS: f-strings or concatenation in text()
result = session.execute(
    text(f"SELECT * FROM users WHERE username = '{user_input}'")  # INJECTION RISK
)
```

**Prisma (Node.js/TypeScript)**:

```typescript
// Safe: Prisma automatically parameterizes
const user = await prisma.user.findUnique({
  where: { email: userInput }
});

// DANGEROUS: Raw queries without parameterization
const result = await prisma.$queryRawUnsafe(
  `SELECT * FROM "User" WHERE email = '${userInput}'`  // INJECTION RISK
);

// Safe: Raw queries with tagged template
const result = await prisma.$queryRaw(
  Prisma.sql`SELECT * FROM "User" WHERE email = ${userInput}`  // Parameterized
);
```

### 28.1.4 Input Validation and Defense in Depth

```python
# Layer 1: Input validation (before database)
import re
from pydantic import BaseModel, validator

class UserQuery(BaseModel):
    user_id: int
    status: str
    
    @validator('status')
    def validate_status(cls, v):
        allowed = {'active', 'inactive', 'pending'}
        if v not in allowed:
            raise ValueError(f'Status must be one of {allowed}')
        return v

# Layer 2: Least privilege database user
-- Database user can only SELECT from specific tables
CREATE USER app_readonly WITH PASSWORD '...';
GRANT SELECT ON TABLE users TO app_readonly;
REVOKE ALL ON TABLE admins FROM app_readonly;

# Layer 3: Query whitelisting (for admin interfaces)
ALLOWED_COLUMNS = {'user_id', 'username', 'email', 'created_at'}
column = request.args.get('sort_by')
if column not in ALLOWED_COLUMNS:
    raise ValueError(f'Invalid sort column: {column}')
```

---

## 28.2 Row-Level Security (RLS)

Row-Level Security enables fine-grained access control where database policies automatically filter rows based on the current user or session variables.

### 28.2.1 RLS Fundamentals

```sql
-- Enable RLS on table (denies all access by default until policies created)
ALTER TABLE documents ENABLE ROW LEVEL SECURITY;

-- Verify RLS status
SELECT relname, relrowsecurity, relforcerowsecurity 
FROM pg_class 
WHERE relname = 'documents';
-- relrowsecurity: true = RLS enabled
-- relforcerowsecurity: true = RLS applies to table owner too

-- By default, table owner bypasses RLS (backwards compatibility)
-- Force owner to obey policies (recommended for defense in depth):
ALTER TABLE documents FORCE ROW LEVEL SECURITY;
```

**Policy Structure**:

```sql
CREATE POLICY policy_name 
ON table_name
[ AS { PERMISSIVE | RESTRICTIVE } ]
[ FOR { ALL | SELECT | INSERT | UPDATE | DELETE } ]
[ TO { role_name | PUBLIC | CURRENT_USER | SESSION_USER } [, ...] ]
[ USING ( using_expression ) ]      -- Applies to SELECT, UPDATE, DELETE
[ WITH CHECK ( check_expression ) ]; -- Applies to INSERT, UPDATE
```

### 28.2.2 Basic RLS Patterns

**Tenant Isolation (Multi-tenant SaaS)**:

```sql
-- Documents table with tenant isolation
CREATE TABLE documents (
    doc_id BIGSERIAL PRIMARY KEY,
    tenant_id BIGINT NOT NULL,
    title TEXT NOT NULL,
    content TEXT,
    owner_id BIGINT NOT NULL,
    is_public BOOLEAN DEFAULT FALSE,
    created_at TIMESTAMPTZ DEFAULT NOW()
);

-- Enable RLS
ALTER TABLE documents ENABLE ROW LEVEL SECURITY;
ALTER TABLE documents FORCE ROW LEVEL SECURITY;

-- Policy: Users can see documents in their tenant
CREATE POLICY tenant_isolation_policy 
ON documents
FOR ALL
TO application_user
USING (tenant_id = current_setting('app.current_tenant_id')::BIGINT)
WITH CHECK (tenant_id = current_setting('app.current_tenant_id')::BIGINT);

-- Application sets tenant context before queries
BEGIN;
SELECT set_config('app.current_tenant_id', '123', false);  -- Set for transaction

-- Now all queries automatically filter to tenant 123
SELECT * FROM documents;  -- Only sees tenant 123's documents

INSERT INTO documents (title, content, tenant_id, owner_id) 
VALUES ('New Doc', 'Content', 123, 456);  -- Allowed (matches policy)

INSERT INTO documents (title, content, tenant_id, owner_id) 
VALUES ('New Doc', 'Content', 999, 456);  -- BLOCKED (violates WITH CHECK)
COMMIT;
```

**User Ownership with Admin Override**:

```sql
-- Users see their own records; admins see all
CREATE TABLE user_notes (
    note_id BIGSERIAL PRIMARY KEY,
    user_id BIGINT REFERENCES users(user_id),
    note_content TEXT,
    is_private BOOLEAN DEFAULT TRUE
);

ALTER TABLE user_notes ENABLE ROW LEVEL SECURITY;

-- Regular users: see own notes or public notes
CREATE POLICY user_notes_access 
ON user_notes
FOR SELECT
TO app_user
USING (
    user_id = current_setting('app.current_user_id')::BIGINT 
    OR is_private = FALSE
);

-- Admin role: see all
CREATE POLICY admin_all_access 
ON user_notes
FOR ALL
TO admin_role
USING (true)
WITH CHECK (true);

-- RESTRICTIVE policy (must pass this AND permissive policies)
CREATE POLICY prevent_cross_tenant 
ON user_notes
AS RESTRICTIVE  -- All restrictive policies must pass
FOR ALL
TO app_user
USING (tenant_id = current_setting('app.current_tenant')::BIGINT);
```

### 28.2.3 Advanced RLS Patterns

**Hierarchical Access (Manager sees subordinates)**:

```sql
CREATE TABLE employees (
    emp_id BIGINT PRIMARY KEY,
    name TEXT,
    manager_id BIGINT REFERENCES employees(emp_id),
    department_id BIGINT,
    salary DECIMAL(10,2)
);

-- Recursive CTE in RLS policy
CREATE POLICY manager_hierarchy 
ON employees
FOR SELECT
TO hr_app
USING (
    emp_id = current_setting('app.current_emp_id')::BIGINT  -- Self
    OR 
    emp_id IN (
        WITH RECURSIVE subordinates AS (
            -- Direct reports
            SELECT emp_id FROM employees 
            WHERE manager_id = current_setting('app.current_emp_id')::BIGINT
            
            UNION ALL
            
            -- Recursive: reports of reports
            SELECT e.emp_id 
            FROM employees e
            JOIN subordinates s ON e.manager_id = s.emp_id
        )
        SELECT emp_id FROM subordinates
    )
);
```

**Time-Based Access (Temporal Policies)**:

```sql
-- Users can only modify documents during business hours in their timezone
CREATE POLICY business_hours_only 
ON documents
FOR UPDATE
TO app_user
USING (
    EXTRACT(HOUR FROM (NOW() AT TIME ZONE current_setting('app.user_timezone'))) 
    BETWEEN 9 AND 17
)
WITH CHECK (
    EXTRACT(HOUR FROM (NOW() AT TIME ZONE current_setting('app.user_timezone'))) 
    BETWEEN 9 AND 17
);
```

### 28.2.4 RLS Performance Optimization

RLS adds predicate overhead to every query. Optimize with proper indexing:

```sql
-- Ensure tenant_id is indexed (critical for RLS performance)
CREATE INDEX idx_documents_tenant_id ON documents(tenant_id);

-- Composite index for common access patterns
CREATE INDEX idx_documents_tenant_owner 
ON documents(tenant_id, owner_id) 
INCLUDE (title, created_at);

-- Check RLS impact with EXPLAIN
EXPLAIN (ANALYZE, BUFFERS) 
SELECT * FROM documents WHERE doc_id = 123;
-- Look for "Rows Removed by Row Level Security Filter" in output
```

**Bypass RLS for Batch Operations**:

```sql
-- Sometimes you need to bypass RLS for maintenance (use with caution)
ALTER TABLE documents DISABLE ROW LEVEL SECURITY;
-- Perform batch operations...
ALTER TABLE documents ENABLE ROW LEVEL SECURITY;

-- Or use SECURITY DEFINER function (runs as function owner)
CREATE OR REPLACE FUNCTION admin_purge_old_docs(cutoff_date DATE)
RETURNS INTEGER AS $$
DECLARE
    deleted_count INTEGER;
BEGIN
    -- Function owner bypasses RLS (unless FORCE RLS is on)
    DELETE FROM documents WHERE created_at < cutoff_date;
    GET DIAGNOSTICS deleted_count = ROW_COUNT;
    RETURN deleted_count;
END;
$$ LANGUAGE plpgsql SECURITY DEFINER;

-- Grant only specific admin users ability to run this
REVOKE ALL ON FUNCTION admin_purge_old_docs FROM PUBLIC;
GRANT EXECUTE ON FUNCTION admin_purge_old_docs TO admin_role;
```

---

## 28.3 Multi-Tenant Security Models

### 28.3.1 Schema-per-Tenant Isolation

**Architecture**: Each tenant gets a dedicated schema with identical table structures.

```sql
-- Template schema with all tables
CREATE SCHEMA template_tenant;

CREATE TABLE template_tenant.users (...);
CREATE TABLE template_tenant.orders (...);
CREATE TABLE template_tenant.documents (...);

-- Provision new tenant
CREATE SCHEMA tenant_123;
-- Copy structure from template
IMPORT FOREIGN SCHEMA template_tenant 
FROM SERVER template_server INTO tenant_123;

-- Or use CREATE TABLE ... LIKE
CREATE TABLE tenant_123.users (LIKE template_tenant.users INCLUDING ALL);
```

**Security Model**:

```sql
-- Tenant-specific user (can only access their schema)
CREATE USER tenant_123_app WITH PASSWORD '...';
GRANT USAGE ON SCHEMA tenant_123 TO tenant_123_app;
GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA tenant_123 TO tenant_123_app;
ALTER DEFAULT PRIVILEGES IN SCHEMA tenant_123 
GRANT SELECT, INSERT, UPDATE, DELETE ON TABLES TO tenant_123_app;

-- Prevent cross-schema access
REVOKE ALL ON SCHEMA tenant_456 FROM tenant_123_app;
```

**Pros**:
- Complete data isolation (tenants cannot query each other even with injection)
- Independent backup/restore per tenant
- Schema migrations can be tenant-specific

**Cons**:
- Connection pooling challenges (must set search_path per connection)
- Higher memory usage (catalog bloat with many schemas)
- Schema changes require applying to all tenant schemas

### 28.3.2 Row-Level Security (Shared Schema)

**Architecture**: Single schema with `tenant_id` columns; RLS enforces isolation.

```sql
-- Shared tables
CREATE TABLE orders (
    order_id BIGSERIAL,
    tenant_id BIGINT NOT NULL,
    customer_id BIGINT,
    total DECIMAL(10,2),
    PRIMARY KEY (tenant_id, order_id)  -- Composite PK for partition pruning
) PARTITION BY LIST (tenant_id);

-- Enable RLS
ALTER TABLE orders ENABLE ROW LEVEL SECURITY;

-- Strict tenant isolation
CREATE POLICY tenant_isolation 
ON orders
FOR ALL
TO app_user
USING (tenant_id = current_setting('app.current_tenant')::BIGINT)
WITH CHECK (tenant_id = current_setting('app.current_tenant')::BIGINT);
```

**Connection Handling**:

```python
# Application sets tenant context per request
def get_db_connection(tenant_id):
    conn = pool.getconn()
    cur = conn.cursor()
    
    # Set tenant context (RLS will enforce)
    cur.execute("SELECT set_config('app.current_tenant', %s, true)", (tenant_id,))
    cur.close()
    
    return conn
```

### 28.3.3 Hybrid Model (Big Tenants Separated)

**Architecture**: Large tenants get dedicated schemas; small tenants share schemas with RLS.

```sql
-- Routing table determines tenant location
CREATE TABLE tenant_routing (
    tenant_id BIGINT PRIMARY KEY,
    tenant_name TEXT,
    storage_type TEXT CHECK (storage_type IN ('dedicated', 'shared')),
    schema_name TEXT,
    shard_id INTEGER
);

-- Application routing logic
def get_tenant_connection(tenant_id):
    routing = db.query(
        "SELECT storage_type, schema_name FROM tenant_routing WHERE tenant_id = %s",
        (tenant_id,)
    )
    
    if routing.storage_type == 'dedicated':
        conn = get_connection(schema=routing.schema_name)
    else:
        conn = get_connection(schema='shared_tenants')
        conn.execute("SELECT set_config('app.current_tenant', %s, true)", (tenant_id,))
    
    return conn
```

---

## 28.4 Data Masking and Anonymization

### 28.4.1 Dynamic Data Masking (PostgreSQL 16+)

PostgreSQL 16 introduces built-in masking through the `GRANT ... MASKED` syntax (or via extensions like `anon` for earlier versions).

```sql
-- Using the anon extension (works on PostgreSQL 12+)
CREATE EXTENSION IF NOT EXISTS anon;

-- Masking rules
SECURITY LABEL FOR anon ON COLUMN users.email 
IS 'MASKED WITH FUNCTION anon.partial_email(email)';

SECURITY LABEL FOR anon ON COLUMN users.phone 
IS 'MASKED WITH FUNCTION anon.partial_phone(phone)';

-- Create masked user
CREATE USER support_staff WITH PASSWORD '...';
GRANT SELECT ON users TO support_staff;

-- When support_staff queries:
SELECT email FROM users LIMIT 1;
-- Returns: a***@example.com instead of alice@example.com
```

**Manual Masking with Views** (Pre-PostgreSQL 16):

```sql
-- Sensitive base table
CREATE TABLE customer_pii (
    customer_id BIGINT PRIMARY KEY,
    full_name TEXT,
    ssn TEXT,  -- Highly sensitive
    email TEXT,
    phone TEXT,
    date_of_birth DATE
);

-- Masked view for general application users
CREATE VIEW customer_safe AS 
SELECT 
    customer_id,
    full_name,
    'XXX-XX-' || RIGHT(ssn, 4) AS ssn_last4,
    regexp_replace(email, '.+@', '***@') AS email_masked,
    regexp_replace(phone, '\d{3}-\d{3}', 'XXX-XXX') AS phone_masked,
    DATE_TRUNC('year', date_of_birth) AS birth_year
FROM customer_pii;

-- Grant access to view, not table
GRANT SELECT ON customer_safe TO app_user;
REVOKE ALL ON customer_pii FROM app_user;

-- Ensure RLS applies to view (if needed)
ALTER VIEW customer_safe SET (security_barrier = true);
```

### 28.4.2 Column-Level Encryption (Application-Side)

For fields requiring encryption at rest with application key management:

```python
# Python: Application-side encryption with Fernet (AES-128)
from cryptography.fernet import Fernet
import base64

class EncryptedField:
    def __init__(self, key):
        self.cipher = Fernet(key)
    
    def encrypt(self, plaintext: str) -> str:
        """Encrypts to URL-safe base64 string"""
        return self.cipher.encrypt(plaintext.encode()).decode()
    
    def decrypt(self, ciphertext: str) -> str:
        """Decrypts from base64 string"""
        return self.cipher.decrypt(ciphertext.encode()).decode()

# Usage
cipher = EncryptedField(os.environ['FIELD_ENCRYPTION_KEY'])

# Before saving to database
encrypted_ssn = cipher.encrypt(user_ssn)  # 'gAAAAABf...'
db.execute("INSERT INTO users (ssn_encrypted) VALUES (%s)", (encrypted_ssn,))

# After retrieval
row = db.execute("SELECT ssn_encrypted FROM users WHERE id = %s", (user_id,))
ssn = cipher.decrypt(row[0])  # '123-45-6789'
```

**Database-Side Encryption (pgcrypto)**:

```sql
-- Using pgcrypto for specific columns
CREATE EXTENSION IF NOT EXISTS pgcrypto;

-- Encrypt on insert (application provides key via session parameter)
INSERT INTO users (email, ssn_encrypted)
VALUES (
    'user@example.com',
    pgp_sym_encrypt('123-45-6789', current_setting('app.encryption_key'))
);

-- Decrypt on select (requires key)
SELECT 
    email,
    pgp_sym_decrypt(ssn_encrypted, current_setting('app.encryption_key')) as ssn
FROM users
WHERE user_id = 123;

-- Without key, returns binary blob (unreadable)
SELECT ssn_encrypted FROM users WHERE user_id = 123;
-- Returns: \x00c04d020...
```

### 28.4.3 Data Redaction for Logs

Prevent sensitive data from appearing in logs:

```sql
-- Mark column as sensitive (log_line_prefix customization)
ALTER SYSTEM SET log_line_prefix = '%t [%p]: [%l-1] user=%u,db=%d,app=%a,client=%h ';
-- Use log_parameter_max_length to prevent logging large parameters
ALTER SYSTEM SET log_parameter_max_length = 128;

-- Application pattern: Never log full queries with parameters
# Bad
logger.info(f"Processing payment for card: {credit_card_number}")

# Good
logger.info(f"Processing payment for user: {user_id}, card ending in: {card_number[-4:]}")
```

**Query Parameter Sanitization**:

```python
# SQLAlchemy event listener to sanitize logs
from sqlalchemy import event

@event.listens_for(Engine, "before_cursor_execute")
def before_cursor_execute(conn, cursor, statement, parameters, context, executemany):
    # Log statement with parameter placeholders only
    logger.debug(f"Executing SQL: {statement}")
    # Never log the 'parameters' tuple if it contains PII
```

---

## 28.5 Security Monitoring and Audit

### 28.5.1 Audit Logging (pgaudit Extension)

```sql
-- Install pgaudit (requires shared_preload_libraries configuration)
CREATE EXTENSION IF NOT EXISTS pgaudit;

-- Configure audit (session level)
SET pgaudit.log = 'write, ddl';
SET pgaudit.log_catalog = off;  -- Don't log catalog queries
SET pgaudit.log_parameter = on; -- Log parameters (sanitized in app layer)

-- Audit specific table
CREATE TABLE sensitive_data (id serial, secret text);

-- All modifications now logged:
-- AUDIT: SESSION,1,1,WRITE,INSERT,TABLE,public.sensitive_data,"INSERT INTO sensitive_data (secret) VALUES ($1)","[redacted]"
```

### 28.5.2 Failed Login Monitoring

```sql
-- Query to check failed authentication attempts
SELECT 
    timestamp,
    user_name,
    database_name,
    client_addr,
    error_severity,
    message
FROM pg_log_files  -- Or external log aggregation
WHERE message LIKE '%password authentication failed%'
   OR message LIKE '%authentication failed%'
ORDER BY timestamp DESC
LIMIT 100;
```

---

## Chapter Summary

In this chapter, you learned:

1. **SQL Injection Prevention**: Always use parameterized queries (prepared statements) where user input is passed separately from SQL commands. Never concatenate user input into query strings. Use `quote_ident()` and `quote_literal()` for dynamic identifiers, and validate inputs against whitelists as defense in depth.

2. **Row-Level Security (RLS)**: Enable fine-grained access control using `ALTER TABLE ... ENABLE ROW LEVEL SECURITY`. Policies use `USING` (for filtering existing rows) and `WITH CHECK` (for validating new/modified rows) clauses. Use `current_setting()` with custom configuration parameters (e.g., `app.current_tenant_id`) to maintain user context across queries without modifying application SQL.

3. **Multi-Tenant Architectures**: Choose between schema-per-tenant (strong isolation, higher overhead), shared-schema with RLS (efficient, requires strict application context management), or hybrid models based on tenant size and isolation requirements. Always use `FORCE ROW LEVEL SECURITY` to ensure superusers and table owners are subject to the same restrictions as application users.

4. **Data Masking**: Implement dynamic data masking using views with `regexp_replace()` for partial redaction, or extensions like `anon` for declarative masking. For highly sensitive fields, use application-side encryption (Fernet/AES) or database-side encryption (`pgp_sym_encrypt` from pgcrypto) with keys managed outside the database (KMS, Vault).

5. **Defense in Depth**: Combine RLS with least-privilege database users (no superuser access for applications), parameterized queries, input validation, and audit logging (pgaudit). Regularly review `pg_stat_statements` and logs for suspicious query patterns, and implement certificate-based authentication for service accounts to eliminate password-related vulnerabilities.

**Next**: In Chapter 29, we will cover Backup and Restore Fundamentals—exploring logical versus physical backup strategies, `pg_dump` and `pg_basebackup` usage, point-in-time recovery architecture, and disaster recovery planning with RPO/RTO objectives.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='27. authentication_pg_hbaconf_and_tls.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../8. reliability_backups_recovery_and_migrations/29. backup_and_restore_fundamentals.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
